## PID + 2D + Drag

### Changes:
- Pass dt into autopilot
- Create function variables Ki and Kd (will need to put them outside the function in the future)
- History of error + need to pass Nstep in autopilot ? and e_prev for the history (needs to be set outside function)
- Changed K_new for K_h (horizontal)and K_h (height) for K_v (vertical)
- c_h = 0
- different error code in autopilot being tested

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive  # what is ipywidgets - slider
from matplotlib import rcParams  # what is rcParams
from numpy.linalg import norm
from numpy.random import randint
import json      #ADDED FOR SAVING FILES
from mpl_toolkits import mplot3d #added for 3D plotting
from scipy.optimize import minimize   # for minimising landing speed
from ipywidgets import interactive 
from matplotlib import rcParams  
from scipy.integrate import quad        # REMOVE IF NOT USED FOR I PART OF PID CONTROL

rcParams['figure.figsize'] = (10, 8)

In [3]:
def mars_surface():
    surfaceN = randint(5, 15)
    land = np.zeros((surfaceN, 2), dtype=int)
    
    # first ensure there's a flat landing site at least 1000m long
    landing_site = randint(1, surfaceN-1)
    land[landing_site, 0] = randint(2000, 5000)
    land[landing_site+1, 0] = min(land[landing_site, 0] + randint(1000, 2000), 6999)
    land[landing_site+1, 1] = land[landing_site, 1] = randint(1, 1500)
    
    # fill in the rest of the terrain
    for i in range(landing_site):
        land[i, 0] = (land[landing_site, 0] / landing_site) * i
        land[i, 1] = randint(0, 1500)
    
    for i in range(landing_site + 2, surfaceN):
        land[i, 0] = (land[landing_site + 1, 0] + 
                      (7000 - land[landing_site + 1, 0]) / len(land[landing_site + 2:]) * 
                      (i - (landing_site + 1)))
        land[i, 1] = randint(0, 1500)
    
    # impose boundary conditions
    land[0, 0] = 0
    land[-1, 0] = 6999

    return land, landing_site

def plot_surface(land, landing_site):
    fig, ax = plt.subplots()
    ax.plot(land[:landing_site+1, 0], land[:landing_site+1, 1], 'k-')
    ax.plot(land[landing_site+1:, 0], land[landing_site+1:, 1], 'k-')
    ax.plot([land[landing_site, 0], land[landing_site+1, 0]], 
             [land[landing_site, 1], land[landing_site+1, 1]], 'k--')
    ax.set_xlim(0, 7000)
    ax.set_ylim(0, 16000) # HERE was 13000
    return ax

def plot_lander(land, landing_site, X, thrust=None, animate=False, step=10):
    if animate:
        def plot_frame(n=len(X)-1):
            ax = plot_surface(land, landing_site)
            ax.plot(X[:n, 0], X[:n, 1], 'b--')
            ax.plot(X[n, 0], X[n, 1], 'b^', ms=20)
            if thrust is not None:
                ax.plot([X[n, 0], X[n, 0] - 100*thrust[n, 0]],
                        [X[n, 1] - 100., X[n, 1] - 100. - 100*thrust[n, 1]], 
                       'r-', lw=10)
        return interactive(plot_frame, n=(0, len(X), step)) #slider
    else:
        ax = plot_surface(land, landing_site) 
        ax.plot(X[:, 0], X[:, 1], 'b--')
        ax.plot(X[-1, 0], X[-1, 1], 'b^')
        return ax

def interpolate_surface(land, x):
    i,  = np.argwhere(land[:, 0] < x)[-1] # segment containing x is [i, i+1]
    m = (land[i+1, 1] - land[i, 1])/(land[i+1, 0] - land[i, 0]) # gradient
    x1, y1 = land[i, :] # point on line with eqn. y - y1 = m(x - x1) 
    return m*(x - x1) + y1

#np.random.seed(20) # seed random number generator for reproducible results
land, landing_site = mars_surface()
#plot_surface(land, landing_site);


def height(land, X):
    return X[1] - interpolate_surface(land, X[0]) #1 in X[1] points to the vertical position y of the lander

assert abs(height(land, [1, land[0, 1]])) < 100.0 # height when on surface left edge should be close to zero
assert abs(height(land, [6999, land[-1, 1]])) < 100.0 # height when on surface at right edge should be close to zero

_land, _landing_site = mars_surface()

## ELIMINATE either above or below

def _height(_land, X):
    return X[1] - interpolate_surface(_land, X[0])

points = np.zeros((10, 2))
points[:, 0] = randint(0, 7000, size=10)
points[:, 1] = randint(0, 16000, size=10) # HERE was 13000
for i in range(10):
    assert abs(height(_land, points[i, :]) - _height(_land, points[i, :])) < 1e-6

In [4]:
g = 3.711 # m/s^2 , gravity on Mars
TSFC = 0.0003 # kg/(N*s)
# fuel in kg
Dc = 6.3525 # drag force as a function of velocity

def simulate(X0, V0, land, landing_site, 
             fuel=400, dt=0.1, Nstep=1000, 
             autopilot=None, print_interval=100, parameters=None, parachute=None):
    
    n = len(X0)       # number of degrees of freedom (2 here)
    X = X0.copy()     # current position
    V = V0.copy()     # current velocity
    Xs = np.zeros((Nstep, n)) # position history (trajectory) 
    Vs = np.zeros((Nstep, n)) # velocity history
    thrust = np.zeros((Nstep, n)) # thrust history
    success = False
    fuel_warning_printed = False
    rotate = randint(-90, 90)            # degrees, initial angle random (heading alignment phase)
    power = 0            # m/s^2, initial thrust power  
    
    e_prev = np.zeros(Nstep) # error history      # HERE

    for i in range(Nstep):
        Xs[i, :] = X     # Store positions
        Vs[i, :] = V     # Store velocities
        
        if autopilot is not None:
            
            rotate, power, parachute = autopilot(i, X, V, fuel, rotate, power, parameters, parachute,dt,e_prev,Nstep)    #HERE 
            assert abs(rotate) <= 90
            assert 0 <= power <= 20000 #changed from 4000, then 12 000
        
            rotate_rad = rotate * np.pi / 180.0 # degrees to radians
            thrust[i, :] = power * np.array([np.sin(rotate_rad), 
                                             np.cos(rotate_rad)])
            if fuel <= 0: 
                if not fuel_warning_printed:
                    print("Fuel empty! Setting thrust to zero")
                    fuel_warning_printed = True
                thrust[i, :] = 0
            else:
                fuel -= TSFC * power * dt
                
        m = 2600 + fuel  #kg , Mass of Lander + Rover + fuel  # fuel Mass loss
        
        if parachute == 0:
            # no Drag
            drag = 0
        else: # parachute == 1
            # Drag - Parachute deployed
            drag = Dc*(V[1]**2)            
        
        A = np.array([0, -g+(drag/m)]) + thrust[i, :]/m
                                   
        V += A * dt                          # update velocities
        X += V * dt                          # update positions
        
        if i % print_interval == 0: 
            print(f"i={i:03d} X=[{X[0]:8.3f} {X[1]:8.3f}] V=[{V[0]:8.3f} {V[1]:8.3f}]"
                  f" thrust=[{thrust[i, 0]:8.3f} {thrust[i, 1]:8.3f}] fuel={fuel:8.3f} rotate={rotate:8.3f} parachute={parachute:8.3f}") 
        
        # check for safe or crash landing
        if X[1] < interpolate_surface(land, X[0]):
            if not (land[landing_site, 0] <= X[0] and X[0] <= land[landing_site + 1, 0]):
                print("crash! did jot land on flat ground!")
            elif rotate != 0: #CHANGE
                print("crash! did not land in a vertical position (tilt angle = 0 degrees)")
            elif abs(V[1]) >= 20: #was 40
                print("crash! vertical speed must be limited (<40m/s in absolute value), got ", abs(V[1]))
            elif abs(V[0]) >= 10: #was 20
                print("crash! horizontal speed must be limited (<20m/s in absolute value), got ", abs(V[0]))
            else:
                print("safe landing - well done!")
                success = True
            Nstep = i
            break
    
    return Xs[:Nstep,:], Vs[:Nstep,:], thrust[:Nstep,:], success, fuel, rotate, parachute

In [8]:
def proportional_autopilot(i, X, V, fuel, rotate, power,parameters, parachute, dt, e_prev, Nstep):
    K_v,K_p,K_h = parameters
    
    K_i = 0.01; # pass them in paramemeters
    K_d = 10; # 10 or 50 seems like it? pass them in paramemeters
    
    c_v = 10.0 # target landing speed, m/s #c vertical
    c_h = 0 #c horizontal and vertical #trade-off for rotation to go back to 0
    
    h = height(land, X)
    
    # Horizontal displacement
    
    Xtarget = (land[landing_site+1, 0] + land[landing_site, 0]) // 2 
    dist = Xtarget-X[0]  #X[i,0], pass the history of X (same for V)
    

    rotate = np.rad2deg(np.arctan2(dist,h))    
      
    # Combine vertical & horizontal errors
    v_target_vert = -(c_v + K_v*h); #FORGOT *h before ??!!  OR now h-2000
    v_target_horz = -K_h*dist; # was: abs(c_h+K_h*dist); not sure if it's actually better or worse
    v_err_vert = abs(v_target_vert - V[1])
    v_err_horz = abs(v_target_horz - V[0])
    e =  v_err_vert + v_err_horz;
    
    e_d = 0
    if i>0:
        e_d = K_d*((e - e_prev[i-1])/dt)
        
    e_prev[i] = e     # Store error
    
    # old: Pout = K_p*e  
    Pout = K_p*(e + e_d + K_i*(e_prev.sum()*dt)) 
    
    power = min(max(Pout, 0.0), 20000.0)
    
    if h > 10000:
        parachute = 0 
    else:
        parachute = 1 #open parachute
    
    
    if i % 100 == 0:
        print(f'e={e:8.3f} Pout={Pout:8.3f} power={power:8.3f}')     
    return (rotate, power, parachute)

## Testing

In [7]:
#np.random.seed(130) # seed random number generator for reproducible results
land, landing_site = mars_surface()

K_v_list = [0.005]
K_h_list = [0.005]
K_p_list = [50]

count = 0
land_speed_results = np.zeros((len(K_v_list),len(K_p_list)))
fuel_results = np.zeros((len(K_v_list),len(K_p_list)))

for i, K_v in enumerate(K_v_list):
    for j, K_p in enumerate(K_p_list):
        for k, K_h in enumerate(K_h_list):
            X0 = [randint(2000, 5000), randint(14000, 16000)] 
            V0 = [randint(-50,50), randint(-500,-100)]  
            #V0[0] = 0 #when using seed() for better results and easier testing of parameters

            Xs, Vs, thrust, success, fuel, rotate, parachute = simulate(X0, V0, land, landing_site, dt=0.1, Nstep=3500, 
                                            autopilot=proportional_autopilot, fuel=400,parameters=[K_v,K_p,K_h],parachute=None)
            land_speed_results[i, j] = Vs[-1,1]
            fuel_results[i, j] = fuel
            count += success

#print(land_speed_results)
#Xtarget = (land[landing_site+1, 0] + land[landing_site, 0]) // 2 
#print(Xtarget)

plot_lander(land, landing_site, Xs, thrust, animate=True, step=10)

e= 208.935 Pout=10447.795 power=10447.795
i=000 X=[3784.703 14138.398] V=[   7.032 -206.024] thrust=[ 964.472 10403.183] fuel= 399.687 rotate=   5.297 parachute=   0.000
e= 213.063 Pout=10943.026 power=10943.026
i=100 X=[3872.421 12071.938] V=[  10.555 -207.052] thrust=[1116.073 10885.964] fuel= 367.240 rotate=   5.854 parachute=   0.000
e=  69.808 Pout=1021.669 power=1021.669
i=200 X=[3989.425 10691.634] V=[  11.980  -62.404] thrust=[ 107.202 1016.029] fuel= 355.060 rotate=   6.023 parachute=   1.000
e=  48.307 Pout=2253.148 power=2253.148
i=300 X=[4112.242 10206.593] V=[  12.629  -41.311] thrust=[ 218.774 2242.501] fuel= 349.362 rotate=   5.572 parachute=   1.000
e=  44.655 Pout=2385.019 power=2385.019
i=400 X=[4242.249 9816.596] V=[  13.358  -37.638] thrust=[ 205.936 2376.111] fuel= 342.350 rotate=   4.953 parachute=   1.000
e=  43.771 Pout=2415.869 power=2415.869
i=500 X=[4379.209 9445.399] V=[  14.012  -36.795] thrust=[ 177.529 2409.337] fuel= 335.139 rotate=   4.214 parachute=   

interactive(children=(IntSlider(value=2728, description='n', max=2729, step=10), Output()), _dom_classes=('wid…

#### Autopilot - error trials

In [ ]:
# From meeting 17: only 1 K for K_h and K_v, representing the trajectory from the x and y coordinates (Height and Dist)

def proportional_autopilot(i, X, V, fuel, rotate, power,parameters, parachute, dt, e_prev, Nstep):
    K_v,K_p,K_h = parameters
    
    K_i = 0.001; # pass them in paramemeters
    K_d = 10; # 10 or 50 seems like it? pass them in paramemeters
    
    c_v = 10.0 # target landing speed, m/s #c vertical
    c_h = 0 #c horizontal and vertical #trade-off for rotation to go back to 0
    
    h = height(land, X)
    
    # Horizontal displacement
    
    Xtarget = (land[landing_site+1, 0] + land[landing_site, 0]) // 2 
    dist = Xtarget-X[0]  #X[i,0], pass the histry of X (same for V)
    

    rotate = np.rad2deg(np.arctan2(dist,h))    
      
    # Combine vertical & horizontal errors
    v_target_vert = -(c_v + K_v*np.sqrt(h**2 + dist**2));
    #v_target_horz = -K_h*dist; # abs(c_h+K_h*dist);
    v_err_vert = abs(v_target_vert - np.sqrt(V[1]**2 + V[0]**2))
    #v_err_horz = abs(v_target_horz - V[0])
    e =  v_err_vert ; #+ v_err_horz;
    
    e_d = 0
    if i>0:
        e_d = K_d*((e - e_prev[i-1])/dt)
        
    e_prev[i] = e     # Store error
    
    # old: Pout = K_p*e  
    Pout = K_p*(e + e_d + K_i*(e_prev.sum()*dt)) 
    
    power = min(max(Pout, 0.0), 20000.0)
    
    if h > 10000:
        parachute = 0 
    else:
        parachute = 1 #open parachute
    
    
    if i % 100 == 0:
        print(f'e={e:8.3f} Pout={Pout:8.3f} power={power:8.3f}')     
    return (rotate, power, parachute)